<a href="https://colab.research.google.com/github/288756/VisArtificial/blob/master/Miniproyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import drive

# Montar el Google Drive en el directorio del proyecto y descomprimir el fichero con los datos
drive.mount('/content/gdrive')
!unzip -n '/content/gdrive/My Drive/vision-artificial.zip' >> /dev/null  # ACTUALIZAR: ruta al fichero comprimido


Mounted at /content/gdrive


In [11]:
# Especificar las rutas al directorio con las imágenes y al fichero con las etiquetas
data_path = '/content/'
imgtrain_dir = data_path + "train/"
csvtrain_file = data_path + "train.csv"

imgtest_dir = data_path + "test/"
csvtest_file = data_path + "test.csv"
# Leer el fichero CSV con las etiquetas
dftrain = pd.read_csv(csvtrain_file, dtype = {"class": "category"})
dftest = pd.read_csv(csvtest_file, dtype = {"class": "category"})

dftrain

,image_filename,mask_filename,class
0,image_003.png,mask_003.png,benign
1,image_004.png,mask_004.png,benign
2,image_008.png,mask_008.png,benign
3,image_012.png,mask_012.png,benign
4,image_016.png,mask_016.png,benign
...,...,...,...
332,image_399.png,mask_399.png,normal
333,image_400.png,mask_400.png,normal
334,image_416.png,mask_416.png,normal
335,image_419.png,mask_419.png,normal


In [5]:
val_size = int(len(dftrain) * 0.2)
dftrain = dftrain.sample(frac=1).reset_index(drop=True)  # barajar el dataframe
dfval = dftrain[:val_size]
print(f'Número de ejemplos del conjunto de entrenamiento: {dftrain.shape[0]}')
print(f'Número de ejemplos del conjunto de validación: {dfval.shape[0]}')
print(f'Número de ejemplos del conjunto de test: {dftest.shape[0]}')

Número de ejemplos del conjunto de entrenamiento: 337
Número de ejemplos del conjunto de validación: 67
Número de ejemplos del conjunto de test: 113


In [12]:
import tensorflow as tf
from tensorflow.data import Dataset

# Dimensiones deseadas de la imagen
img_width, img_height = 224, 224
n_channels = 3                # número de canales (RGB)
n_classes = 5                 # número de clases
x_col = 'image_filename'      # nombres de las columnas en el fichero CSV
y_col = 'class'

# Cargar y preprocesar imágenes
def load_and_preprocess_image(image_filename, label_str):
    image_path = tf.strings.join([imgtrain_dir, image_filename])
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=n_channels)
    # Redimensionar la imagen al tamaño deseado con relleno de ceros si es necesario
    image = tf.image.resize_with_pad(image, img_width, img_height)
    image = image / 255.0                               # normalización
    label = label_str                                     # No es necesario convertir a número si las etiquetas son strings
    return image, label

# Crear conjunto de datos
def get_dataset(df):
    image_filenames = df[x_col].values
    labels = df[y_col].values
    dataset = tf.data.Dataset.from_tensor_slices((image_filenames, labels))
    dataset = dataset.map(load_and_preprocess_image)
    return dataset

# Crear los conjuntos de datos y preparar los lotes
batch_size = 128
train_dataset = get_dataset(dftrain).batch(batch_size)
val_dataset = get_dataset(dfval).batch(batch_size)
test_dataset = get_dataset(dftest).batch(batch_size)

print(f'Número de lotes del conjunto de entrenamiento: {len(train_dataset)}')
print(f'Número de lotes del conjunto de validación: {len(val_dataset)}')
print(f'Número de lotes del conjunto de test: {len(test_dataset)}')

KeyError: 'class'

In [ ]:
# Definir el modelo de red neuronal convolucional (CNN)
def create_model(input_shape):
    inputs_image = layers.Input(shape=input_shape)
    inputs_mask = layers.Input(shape=input_shape)

    # Procesamiento de la imagen
    conv1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs_image)
    conv2 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D((2, 2))(conv2)

    # Procesamiento de la máscara
    conv3 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs_mask)
    conv4 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(conv3)
    pool2 = layers.MaxPooling2D((2, 2))(conv4)

    # Combinar características de la imagen y la máscara
    merged = layers.concatenate([pool1, pool2])

    # Más capas convolucionales y de pooling...

    # Capa de salida
    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(merged)

    model = models.Model(inputs=[inputs_image, inputs_mask], outputs=outputs)
    return model

# Crear el modelo
input_shape = (224, 224, 3)  # Suponiendo que las imágenes son de tamaño 224x224 y tienen 3 canales
model = create_model(input_shape)

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo (suponiendo que tienes tus datos de entrenamiento en X_train_images, X_train_masks, y tus etiquetas en y_train)
model.fit([X_train_images, X_train_masks], y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluar el modelo
model.evaluate([X_test_images, X_test_masks], y_test)